In [1]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = 10
T = 100
correlation_List = [0.4]

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

# Paramètres pour la simulation
num_samples = 16290  # Nombre d'échantillons à générer
L = 10  # Nombre de capteurs
T = 100  # Nombre de points temporels
phi_max = 90  # φmax
rho = 1  # Résolution
G = np.arange(-phi_max, phi_max + rho, rho)


In [5]:
import numpy as np
from numpy.linalg import eigh

def generate_music_spectrum(R, L, G, d=0.5, fs=1):
    """
    Calcule le spectre MUSIC à partir de la matrice de covariance R.

    :param R: Matrice de covariance estimée.
    :param L: Nombre de capteurs.
    :param G: Grille des angles (en degrés).
    :param d: Distance entre les éléments du réseau de capteurs (en longueurs d'onde).
    :param fs: Fréquence d'échantillonnage.
    :return: Spectre MUSIC pour les angles dans G.
    """
    # Calcul des valeurs propres et vecteurs propres
    eigvals, eigvecs = eigh(R)
    
    # Séparation des vecteurs du signal et du bruit
    E_noise = eigvecs[:, :L-2]  # Supposant 2 sources max
    
    # Calcul du spectre MUSIC
    P_music = np.zeros(len(G))
    for i, theta in enumerate(G):
        a = np.exp(np.arange(L) * 1j * 2 * np.pi * d * np.sin(np.radians(theta)) / fs)
        P_music[i] = 1 / np.abs(a.conj().T @ E_noise @ E_noise.conj().T @ a)
    
    # Normalisation du spectre
    P_music = P_music / np.max(P_music)
    
    return P_music

# Initialisation des tableaux pour les données
all_R_hat_with_phase = np.zeros((num_samples, 3, L, L))


for i in range(num_samples):
    # Générer les angles d'arrivée et la matrice X comme décrit précédemment
    thetaList = [np.random.uniform(-90, 90) for _ in range(2)]  # Exemple de génération d'angles
    varList = [np.random.uniform(0, 1000000000000) for _ in range(nbSources)]
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)  # Votre fonction pour générer X, ajoutez les paramètres nécessaires
    
    # Calculer R_hat étendue avec phase
    R_hat_with_phase = generate_R_hat_with_phase(X)
    all_R_hat_with_phase[i] = R_hat_with_phase
    


# Utilisation de la fonction generate_music_spectrum sur vos données
all_music_spectra = np.zeros((num_samples, len(G)))
for i in range(num_samples):
    R = all_R_hat_with_phase[i, :, :, 0] + 1j * all_R_hat_with_phase[i, :, :, 1]  # Reconstitution de la matrice complexe
    all_music_spectra[i] = generate_music_spectrum(R, L, G)

# Division des données en ensembles d'entraînement et de test
X_train, X_temp, Y_train, Y_temp = train_test_split(all_R_hat_with_phase, all_music_spectra, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

[[5.01147002e+11 1.27641682e+11]
 [1.27641682e+11 2.03188872e+11]]
[[4.99238009e+11 2.57016599e+11]
 [2.57016599e+11 8.26979454e+11]]
[[6.69967323e+11 6.96058692e+10]
 [6.96058692e+10 4.51978855e+10]]
[[5.26782285e+11 1.14411938e+11]
 [1.14411938e+11 1.55307182e+11]]
[[3.77555312e+11 1.00222456e+11]
 [1.00222456e+11 1.66275979e+11]]
[[3.44948727e+11 1.56618156e+11]
 [1.56618156e+11 4.44436461e+11]]
[[5.35367041e+11 2.54953482e+11]
 [2.54953482e+11 7.58840118e+11]]
[[2.44709923e+11 1.33429506e+11]
 [1.33429506e+11 4.54707579e+11]]
[[8.13991175e+11 2.93702619e+11]
 [2.93702619e+11 6.62332336e+11]]
[[8.18321053e+11 2.30799668e+11]
 [2.30799668e+11 4.06842818e+11]]
[[6.95601461e+11 3.09679544e+11]
 [3.09679544e+11 8.61677135e+11]]
[[2.34183867e+11 1.77287428e+11]
 [1.77287428e+11 8.38839598e+11]]
[[2.49579678e+11 9.41714380e+10]
 [9.41714380e+10 2.22079874e+11]]
[[3.98446091e+11 1.89857994e+11]
 [1.89857994e+11 5.65416169e+11]]
[[3.80455825e+11 2.44584814e+11]
 [2.44584814e+11 9.82731224e+

LinAlgError: Last 2 dimensions of the array must be square

In [ ]:
class DOADataset(Dataset):
    def __init__(self, covariance_matrices, spectra):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - spectra (np.ndarray): Spectres MUSIC sous forme de vecteurs.
          Forme attendue: (nb_samples, len(G)), où len(G) est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.spectra = spectra

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
        cov_matrix = self.covariance_matrices[idx]
        spectrum = self.spectra[idx]

        # Conversion en tenseur PyTorch
        cov_matrix = torch.tensor(cov_matrix, dtype=torch.float32)
        spectrum = torch.tensor(spectrum, dtype=torch.float32)

        return cov_matrix, spectrum

# Utilisation avec vos données X_train et Y_train
train_dataset = DOADataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Utilisation avec vos données X_val et Y_val pour l'ensemble de validation
val_dataset = DOADataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # Pas besoin de shuffle pour l'ensemble de validation

# Utilisation avec vos données X_test et Y_test pour l'ensemble de test
test_dataset = DOADataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

import torch.nn as nn

class DeepMusicModel(nn.Module):
    def __init__(self, num_classes=181):
        super(DeepMusicModel, self).__init__()

        # Couches convolutionnelles
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        # Couches d'activation ReLU
        self.relu2 = nn.ReLU()
        self.relu5 = nn.ReLU()
        self.relu8 = nn.ReLU()
        self.relu11 = nn.ReLU()

        # Couches de normalisation
        self.norm3 = nn.BatchNorm2d(256)
        self.norm6 = nn.BatchNorm2d(256)
        self.norm9 = nn.BatchNorm2d(256)
        self.norm12 = nn.BatchNorm2d(256)

        # Couche de convolution (kernel 1x1)
        self.conv15 = nn.Conv2d(256, 256, kernel_size=1)

        # Couche de dropout
        self.dropout16 = nn.Dropout(0.5)

        # Couches entièrement connectées (FC)
        self.fc17 = nn.Linear(in_features=7680, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu2(x)
        x = self.norm3(x)
        
        x = self.conv4(x)
        x = self.relu5(x)
        x = self.norm6(x)
        
        x = self.conv7(x)
        x = self.relu8(x)
        x = self.norm9(x)
        
        x = self.conv10(x)
        x = self.relu11(x)
        x = self.norm12(x)
        
        x = self.conv13(x)
        x = self.conv15(x)
        
        x = self.dropout16(x)
        x = x.view(x.size(0), -1)  # Aplatir pour FC
        x = self.fc17(x)
        

        return x


# Création de l'instance du modèle
model = DeepMusicModel()

# Définition de l'optimiseur et de la fonction de perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


# Définir le nombre d'époques
num_epochs = 10

for epoch in range(num_epochs):
    # Entraînement
    model.train()
    total_loss = 0.0
    for cov_matrices, doa_labels in train_loader:
        optimizer.zero_grad()

        outputs = model(cov_matrices)
        loss = criterion(outputs, doa_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}')

Epoch 1, Training Loss: 38.05885510351143
Epoch 2, Training Loss: 36.626569598328835
Epoch 3, Training Loss: 36.3034839443132
Epoch 4, Training Loss: 36.127083469839654
Epoch 5, Training Loss: 36.02589724110622
Epoch 6, Training Loss: 35.96395684223549
Epoch 7, Training Loss: 35.9105419551625
Epoch 8, Training Loss: 35.902802093356264
Epoch 9, Training Loss: 35.85554594152114
Epoch 10, Training Loss: 35.83898687362671
